In [61]:
import pandas as pd
import os,h5py
import pandas as pd
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict
%matplotlib inline  

os.chdir('/data/arpit.goel/20_TwoSigmaFinancial')
df = pd.read_hdf('01.RawData/train.h5')

In [45]:
slow_moving_columns=['fundamental_22','fundamental_49','technical_14','technical_43','technical_17',\
                    'technical_2','technical_6','technical_29','technical_11','fundamental_20','fundamental_16',\
                    'technical_10','fundamental_14','fundamental_59','fundamental_46','fundamental_9',\
                    'fundamental_24','fundamental_12','fundamental_52','fundamental_3','fundamental_39',\
                    'technical_20','technical_34','fundamental_40','fundamental_43','technical_30',\
                    'technical_12','derived_1','fundamental_31','fundamental_37','technical_13',\
                    'fundamental_21','fundamental_58','fundamental_28','fundamental_51','technical_7',\
                    'fundamental_1','fundamental_56','technical_0','fundamental_35','fundamental_54',\
                    'fundamental_23','fundamental_45','technical_37','fundamental_63','technical_32',\
                    'fundamental_50','technical_38','fundamental_53','fundamental_57','technical_39']


df = df[slow_moving_columns+['y','timestamp','id']]
df_median = df.sample(n=100000).median(axis=0)
df=df.fillna(df_median)

In [46]:
df['timestamp_new']=np.floor(df['timestamp']/30)
df['y_new']=np.log(1+df['y'])
df_rolled=df.groupby(['id','timestamp_new'])[slow_moving_columns+['y','y_new']].mean().reset_index()


In [55]:
pred_vars=df_rolled[slow_moving_columns].corrwith(df_rolled['y_new']).abs().sort_values(ascending=False)
pred_vars=pred_vars[pred_vars>0.05].index.tolist()

df_rolled[pred_vars].corr()


,technical_30,technical_20,technical_17,technical_2,technical_38,technical_11,technical_39,technical_12,technical_32,technical_13,technical_37,technical_6
technical_30,1.000000,-0.220216,-0.209567,-0.247090,0.144514,-0.264315,0.121681,0.129073,0.127337,-0.251023,0.099304,-0.240112
technical_20,-0.220216,1.000000,0.103378,0.131822,-0.138915,0.104504,-0.127072,-0.119112,-0.133501,-0.181409,-0.091144,0.022740
technical_17,-0.209567,0.103378,1.000000,0.559390,-0.281501,0.352378,-0.258635,-0.381435,-0.250723,0.066160,-0.289884,0.226983
technical_2,-0.247090,0.131822,0.559390,1.000000,-0.291237,0.608621,-0.261321,-0.383196,-0.282122,0.027543,-0.287789,0.396557
technical_38,0.144514,-0.138915,-0.281501,-0.291237,1.000000,-0.330985,0.587471,0.344578,0.406914,0.043821,0.457957,-0.341225
technical_11,-0.264315,0.104504,0.352378,0.608621,-0.330985,1.000000,-0.289464,-0.290468,-0.302923,0.007131,-0.294702,0.640128
technical_39,0.121681,-0.127072,-0.258635,-0.261321,0.587471,-0.289464,1.000000,0.447013,0.509262,0.049934,0.446051,-0.230580
technical_12,0.129073,-0.119112,-0.381435,-0.383196,0.344578,-0.290468,0.447013,1.000000,0.401460,0.045694,0.293348,-0.165946
technical_32,0.127337,-0.133501,-0.250723,-0.282122,0.406914,-0.302923,0.509262,0.401460,1.000000,0.066762,0.176281,-0.268051
technical_13,-0.251023,-0.181409,0.066160,0.027543,0.043821,0.007131,0.049934,0.045694,0.066762,1.000000,0.016013,-0.031301


In [56]:
from sklearn.linear_model import LinearRegression, Ridge, ElasticNetCV

train=pd.read_pickle('03.ProcessedData/train_with_predicts_ids.pkl').sample(n=100000)
test_id=pd.read_pickle('03.ProcessedData/test_id_with_predicts_ids.pkl')
test_ts=pd.read_pickle('03.ProcessedData/test_ts_with_predicts_ids.pkl')

msk = np.random.rand(len(train)) < 0.6
ins = train[msk]
oos = train[~msk]

ins['y_new']=np.log(1+ins['y'])
oos['y_new']=np.log(1+oos['y'])
test_id['y_new']=np.log(1+test_id['y'])
test_ts['y_new']=np.log(1+test_ts['y'])


/data/arpit.goel/anacondainstall/lib/python2.7/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/data/arpit.goel/anacondainstall/lib/python2.7/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [59]:
cols=['technical_13','technical_17','technical_20','technical_30']

gb = LinearRegression()
gb.fit(df_rolled[cols],df_rolled['y_new'])
print gb.score(df_rolled[cols],df_rolled['y_new'])
print gb.score(ins[cols],ins['y_new']), gb.score(oos[cols],oos['y_new'])
print gb.score(test_ts[cols],test_ts['y_new']), gb.score(test_id[cols],test_id['y_new'])


0.416715637114
-0.0316790338252 -0.0296740426774
-0.0259102556528 -0.0291972968983


In [60]:
exp=df[['y']].head()
exp['y_new']=np.log(1+exp['y'])
exp['y_der']=np.exp(exp['y_new'])-1
exp

,y,y_new,y_der
0,-0.011753,-0.011823,-0.011753
1,-0.001240,-0.001241,-0.001240
2,-0.020940,-0.021162,-0.020940
3,-0.015959,-0.016088,-0.015959
4,-0.007338,-0.007365,-0.007338


In [62]:
market_df=df.groupby('timestamp').mean()
market_df.corrwith(market_df['y']).abs().sort_values().tail(20)

fundamental_34    0.044809
technical_36      0.045049
technical_0       0.046533
technical_19      0.046765
technical_35      0.049074
fundamental_53    0.054306
technical_14      0.054979
technical_29      0.055054
technical_10      0.056044
fundamental_11    0.056250
technical_27      0.058622
fundamental_57    0.058809
fundamental_18    0.062293
fundamental_29    0.069110
fundamental_55    0.076870
fundamental_60    0.078106
fundamental_56    0.078286
fundamental_62    0.081237
fundamental_15    0.084768
y                 1.000000
dtype: float64